In [50]:
# Import the modules
import pandas as pd
from datetime import datetime

# Create the list of data files to loop through to read in and process
start_year = 2021
csv_list = ['201809-citibike-tripdata.csv', 
            '202009-citibike-tripdata.csv', 
            '202309-citibike-tripdata.csv']
#for year in range(start_year, 2024):
#    csv_list.append(str(year) + '09' + '-citibike-tripdata.csv')

#################################################################

#  DataFrame used to accumulate stats from each month        
September_df = pd.DataFrame()

#################################################################

# the following lists are used to account for various naming conventions across the datasets
start_column_names_list = ['starttime', 'started_at', 'Start Time']
stop_column_names_list = ['stoptime', 'ended_at', 'Stop Time']
usertype_column_names_list = ['usertype', 'member_casual', 'User Type']
duration_column_names_list = ['tripduration', 'Trip Duration']
start_lng_column_names_list = ['start_lng', 'start station longitude']
start_lat_column_names_list = ['start_lat', 'start station latitude']
start_station_column_names_list = ['start_station_name', 'start station name']



# the following are the chosen convention for names of the variables of interest
start_time_chosen_name = 'starttime'
stop_time_chosen_name = 'stoptime'
user_type_chosen_name = 'usertype'
duration_chosen_name = 'tripduration'

#################################################################
#    Function that standardizes the following:
#        'starttime' column
#        'stoptime' column
#        'usertype' column - it will standardize the user type to 'member' or 'casual'
#        'tripduration' column - it wil create this column if not in the dataset       

def standardize_data(df):
       
    for i in range(0, len(start_column_names_list)):
        if start_column_names_list[i] in df.columns:
            df = df.rename(columns={start_column_names_list[i]: 'starttime'})

    for i in range(0, len(stop_column_names_list)):
        if stop_column_names_list[i] in df.columns:
            df = df.rename(columns={stop_column_names_list[i]: 'stoptime'})

    for i in range(0, len(usertype_column_names_list)):
        if usertype_column_names_list[i] in df.columns:
            df = df.rename(columns={usertype_column_names_list[i]: 'usertype'})
            
    for i in range(0, len(start_lng_column_names_list)):
        if start_lng_column_names_list[i] in df.columns:
            df = df.rename(columns={start_lng_column_names_list[i]: 'start longitude'})

    for i in range(0, len(start_lat_column_names_list)):
        if start_lat_column_names_list[i] in df.columns:
            df = df.rename(columns={start_lat_column_names_list[i]: 'start latitude'})
   
    for i in range(0, len(start_station_column_names_list)):
        if start_station_column_names_list[i] in df.columns:
            df = df.rename(columns={start_station_column_names_list[i]: 'start station name'})
   

    # change 'Subscriber' to 'Member' and 'Customer' to 'Casual' if needed
    if df['usertype'][0] == 'Subscriber' or df['usertype'][0] == 'Customer':    
        df.loc[df['usertype'] == 'Subscriber', 'usertype'] = 'member'
        df.loc[df['usertype'] == 'Customer', 'usertype'] = 'casual' 

    # correctly name the trip duration column if it exists; create duration column if it does not exist    
    duration_column_exists = 0    
    for i in range(0, len(duration_column_names_list)):
        if duration_column_names_list[i] in df.columns:
            df = df.rename(columns={duration_column_names_list[i]: duration_chosen_name})
            duration_column_exists = 1
    # if duration column does not exist, calculate and create one
    if duration_column_exists == 0:
        df[start_time_chosen_name] = pd.to_datetime(df[start_time_chosen_name])
        df[stop_time_chosen_name] = pd.to_datetime(df[stop_time_chosen_name])
        df[duration_chosen_name] = (df[stop_time_chosen_name] - df[start_time_chosen_name]) / pd.Timedelta(seconds=1)
        
    return df

#################################################################
# 

#def create_september_df(df): 
#    # Eliminate the time portion of datetime
#    df['Start Day'] = pd.to_datetime(df[start_time_chosen_name]).dt.date
#    
#    # Choose only the type of user, the start day, and the trip duration
#    narrowed_df = df[[user_type_chosen_name, 'Start Day', duration_chosen_name]]
#
#    # Loop through each start day and gather counts of member trips, 
#    # casual trips, and duration of member and casual trips
#    day_member_count_list = []
#    day_casual_count_list = []
#    duration_member_ave_list = []
#    duration_casual_ave_list = []
#    
#    # create the day list - this will be 28, 29, 30, or 31 in length, based on the data for that month
#    day_list = narrowed_df['Start Day'].unique()
#
#    for i in range(0, len(day_list)):
#        # count the number of member trips and casual trips for each day
#        member_count = len(narrowed_df[(narrowed_df[user_type_chosen_name] == 'member') & (narrowed_df['Start Day'] == day_list[i])])        
#        casual_count = len(narrowed_df[(narrowed_df[user_type_chosen_name] == 'casual') & (narrowed_df['Start Day'] == day_list[i])])
#        
#        # find the average trip duration for members and casuals
#        duration_member_ave = narrowed_df.loc[(narrowed_df[user_type_chosen_name] == 'member') & (narrowed_df['Start Day'] == day_list[i]), duration_chosen_name].mean()
#        duration_casual_ave = narrowed_df.loc[(narrowed_df[user_type_chosen_name] == 'casual') & (narrowed_df['Start Day'] == day_list[i]), duration_chosen_name].mean()
#        
#        # populate the lists of counts and averages
#        day_member_count_list.append(member_count)
#        day_casual_count_list.append(casual_count)
#        
#        # save the duration in minutes
#        duration_member_ave_list.append(round(duration_member_ave/60, 2))
#        duration_casual_ave_list.append(round(duration_casual_ave/60, 2))
#                                      
#    # create the new df of stats, including the start date
#    stats = pd.DataFrame({'Start Day': day_list,
#                          'Member Count': day_member_count_list,
#                          'Casual Count': day_casual_count_list,
#                          'Member Duration Avg': duration_member_ave_list,
#                          'Casual Duration Avg': duration_casual_ave_list
#                         })
#    return stats
    
##############################################################
# main code to loop through all of the months of data in csv_list

for dataset in range(0, len(csv_list)):
    
    # Bring in one csv file at a time
    file_path = csv_list[dataset]
    
    # Read the csv file into a DataFrame
    ### warning errors ocurred concerning mismatching data types and suggested using 'low_memory=False'
    # by_month_df = pd.read_csv(file_path)
    september_df = pd.read_csv(file_path, low_memory=False)
 
    # call function to standardize column names and values across different formats
    standard_df = standardize_data(september_df)
    
    # narrow the df to only necessary data
    standard_df = standard_df[['tripduration', 'starttime', 'start station name', 'start latitude', 'start longitude', 'usertype']]
    
    standard_df['starttime'] = pd.to_datetime(standard_df['starttime'])
    
#    standard_df['starttime'] = pd.to_datetime(standard_df['starttime'], format='%m/%d/%Y %H:%M:%S')

    #standard_df['stoptime'] = pd.to_datetime(standard_df['stoptime'], format='%m/%d/%Y %H:%M:%S')
    
    #standard_df['duration'] = round((standard_df['duration'] pd.Timedelta(minutes=1), 2)
    
    # Accumulate/concatenate the monthly stats into a dataframe
    September_df = pd.concat([September_df, standard_df], ignore_index=True)

In [53]:
September_df.tail()

,tripduration,starttime,start station name,start latitude,start longitude,usertype
7941266,753.0,2023-09-28 18:46:55,Rogers Ave & Sterling St,40.663062,-73.953875,member
7941267,605.0,2023-09-12 08:13:58,Prospect Pl & Underhill Ave,40.676969,-73.965790,member
7941268,987.0,2023-09-11 16:48:06,E 56 St & Madison Ave,40.761573,-73.972628,member
7941269,407.0,2023-09-07 07:42:11,E 77 St & 3 Ave,40.773280,-73.958971,member
7941270,503.0,2023-09-22 13:04:02,Broadway & W 58 St,40.766953,-73.981693,member


In [54]:
September_df.dtypes

tripduration                 float64
starttime             datetime64[ns]
start station name            object
start latitude               float64
start longitude              float64
usertype                      object
dtype: object

In [55]:
September_df['tripduration'] = September_df['tripduration'].astype(int)

In [56]:
September_df.dtypes

tripduration                   int32
starttime             datetime64[ns]
start station name            object
start latitude               float64
start longitude              float64
usertype                      object
dtype: object

In [57]:
September_sorted_df = September_df.sort_values(['starttime'])
September_sorted_df.tail()

,tripduration,starttime,start station name,start latitude,start longitude,usertype
7210125,2193,2023-09-30 23:59:55,1 Ave & E 110 St,40.792511,-73.938289,member
6215998,642,2023-09-30 23:59:55,Myrtle Ave & Marcy Ave,40.695392,-73.949788,member
5222698,387,2023-09-30 23:59:55,Lispenard St & Broadway,40.719392,-74.002472,casual
5247222,664,2023-09-30 23:59:57,Broadway & W 29 St,40.746201,-73.988557,casual
4828427,335,2023-09-30 23:59:58,5 Ave & E 72 St,40.772613,-73.967067,casual


In [58]:
# Finally, save the full stats file to csv
September_sorted_df.to_csv('september_2018_2020_2023.csv', index=False)

In [26]:
 # Bring in one csv file at a time
file_path ='201709-citibike-tripdata.csv'

# Read the csv file into a DataFrame
### warning errors ocurred concerning mismatching data types and suggested using 'low_memory=False'
# by_month_df = pd.read_csv(file_path)
show_df = pd.read_csv(file_path, low_memory=False)
show_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,362,2017-09-01 00:00:17,2017-09-01 00:06:19,3331,Riverside Dr & W 104 St,40.801343,-73.971146,3328,W 100 St & Manhattan Ave,40.795000,-73.964500,14530,Subscriber,1993.0,1
1,188,2017-09-01 00:00:21,2017-09-01 00:03:30,3101,N 12 St & Bedford Ave,40.720798,-73.954847,3100,Nassau Ave & Newell St,40.724813,-73.947526,15475,Subscriber,1988.0,1
2,305,2017-09-01 00:00:25,2017-09-01 00:05:30,3140,1 Ave & E 78 St,40.771404,-73.953517,3141,1 Ave & E 68 St,40.765005,-73.958185,30346,Subscriber,1969.0,1
3,223,2017-09-01 00:00:52,2017-09-01 00:04:36,236,St Marks Pl & 2 Ave,40.728419,-73.987140,473,Rivington St & Chrystie St,40.721101,-73.991925,28056,Subscriber,1993.0,1
4,758,2017-09-01 00:01:01,2017-09-01 00:13:40,3427,Lafayette St & Jersey St,40.724305,-73.996010,3431,E 35 St & 3 Ave,40.746524,-73.977885,25413,Subscriber,1987.0,1


In [27]:
show_df.dtypes

tripduration                 int64
starttime                   object
stoptime                    object
start station id             int64
start station name          object
start station latitude     float64
start station longitude    float64
end station id               int64
end station name            object
end station latitude       float64
end station longitude      float64
bikeid                       int64
usertype                    object
birth year                 float64
gender                       int64
dtype: object

In [24]:
file_path ='201409-citibike-tripdata.csv'

# Read the csv file into a DataFrame
### warning errors ocurred concerning mismatching data types and suggested using 'low_memory=False'
# by_month_df = pd.read_csv(file_path)
show_df = pd.read_csv(file_path, low_memory=False)
show_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,2828,9/1/2014 00:00:25,9/1/2014 00:47:33,386,Centre St & Worth St,40.714948,-74.002345,450,W 49 St & 8 Ave,40.762272,-73.987882,15941,Subscriber,1980.0,1
1,368,9/1/2014 00:00:28,9/1/2014 00:06:36,387,Centre St & Chambers St,40.712733,-74.004607,2008,Little West St & 1 Pl,40.705693,-74.016777,18962,Subscriber,1982.0,1
2,2201,9/1/2014 00:00:40,9/1/2014 00:37:21,386,Centre St & Worth St,40.714948,-74.002345,441,E 52 St & 2 Ave,40.756014,-73.967416,15982,Subscriber,1968.0,1
3,322,9/1/2014 00:00:41,9/1/2014 00:06:03,167,E 39 St & 3 Ave,40.748901,-73.976049,528,2 Ave & E 31 St,40.742909,-73.977061,19081,Subscriber,1961.0,1
4,1693,9/1/2014 00:00:59,9/1/2014 00:29:12,223,W 13 St & 7 Ave,40.737815,-73.999947,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,20836,Subscriber,1978.0,1


In [25]:
show_df.dtypes

tripduration                 int64
starttime                   object
stoptime                    object
start station id             int64
start station name          object
start station latitude     float64
start station longitude    float64
end station id               int64
end station name            object
end station latitude       float64
end station longitude      float64
bikeid                       int64
usertype                    object
birth year                 float64
gender                       int64
dtype: object

In [29]:
file_path ='data_2014_to_2023.csv'

# Read the csv file into a DataFrame
### warning errors ocurred concerning mismatching data types and suggested using 'low_memory=False'
# by_month_df = pd.read_csv(file_path)
show_df = pd.read_csv(file_path, low_memory=False)
show_df.head() 

,Start Day,Member Count,Casual Count,Member Duration Avg,Casual Duration Avg
0,2014-01-01,5407,652,11.66,46.25
1,2014-01-02,8419,181,12.79,21.91
2,2014-01-03,1123,21,13.33,16.36
3,2014-01-04,2263,29,20.29,28.75
4,2014-01-05,2643,35,13.61,23.73


In [30]:
show_df.dtypes

Start Day               object
Member Count             int64
Casual Count             int64
Member Duration Avg    float64
Casual Duration Avg    float64
dtype: object

In [4]:
start_year = 2014
csv_list = []
for year in range(start_year, 2024):
    csv_list.append(str(year) + '09' + '-citibike-tripdata.csv')
print(csv_list)

['201409-citibike-tripdata.csv', '201509-citibike-tripdata.csv', '201609-citibike-tripdata.csv', '201709-citibike-tripdata.csv', '201809-citibike-tripdata.csv', '201909-citibike-tripdata.csv', '202009-citibike-tripdata.csv', '202109-citibike-tripdata.csv', '202209-citibike-tripdata.csv', '202309-citibike-tripdata.csv']


In [3]:
df['starttime'] = pd.to_datetime(df['starttime'], format='%m/%d/%Y %H:%M:%S')
df['stoptime'] = pd.to_datetime(df['stoptime'], format='%m/%d/%Y %H:%M:%S')

In [62]:
df['duration'] = round((df['stoptime'] - df['starttime']) / pd.Timedelta(minutes=1), 2)
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,duration
0,538,2016-05-01 00:00:03,2016-05-01 00:09:02,536,1 Ave & E 30 St,40.741444,-73.975361,497,E 17 St & Broadway,40.737050,-73.990093,23097,Subscriber,1986.0,2,8.98
1,224,2016-05-01 00:00:04,2016-05-01 00:03:49,361,Allen St & Hester St,40.716059,-73.991908,340,Madison St & Clinton St,40.712690,-73.987763,23631,Subscriber,1977.0,1,3.75
2,328,2016-05-01 00:00:14,2016-05-01 00:05:43,301,E 2 St & Avenue B,40.722174,-73.983688,311,Norfolk St & Broome St,40.717227,-73.988021,23049,Subscriber,1980.0,1,5.48
3,1196,2016-05-01 00:00:20,2016-05-01 00:20:17,3141,1 Ave & E 68 St,40.765005,-73.958185,237,E 11 St & 2 Ave,40.730473,-73.986724,19019,Customer,NaN,0,19.95
4,753,2016-05-01 00:00:26,2016-05-01 00:13:00,492,W 33 St & 7 Ave,40.750200,-73.990931,228,E 48 St & 3 Ave,40.754601,-73.971879,16437,Subscriber,1981.0,1,12.57


In [66]:
df = df[['usertype', 'starttime', 'duration', 'start station name', 'start station latitude', 'start station longitude']]
df.head()

,usertype,starttime,duration,start station name,start station latitude,start station longitude,end station name,end station latitude,end station longitude
0,Subscriber,2016-05-01 00:00:03,8.98,1 Ave & E 30 St,40.741444,-73.975361,E 17 St & Broadway,40.737050,-73.990093
1,Subscriber,2016-05-01 00:00:04,3.75,Allen St & Hester St,40.716059,-73.991908,Madison St & Clinton St,40.712690,-73.987763
2,Subscriber,2016-05-01 00:00:14,5.48,E 2 St & Avenue B,40.722174,-73.983688,Norfolk St & Broome St,40.717227,-73.988021
3,Customer,2016-05-01 00:00:20,19.95,1 Ave & E 68 St,40.765005,-73.958185,E 11 St & 2 Ave,40.730473,-73.986724
4,Subscriber,2016-05-01 00:00:26,12.57,W 33 St & 7 Ave,40.750200,-73.990931,E 48 St & 3 Ave,40.754601,-73.971879


In [86]:
narrowed_df = sorted_df[['usertype', 'starttime', 'duration', 'start station name', 'start station latitude', 'start station longitude']]
narrowed_df.head()

,usertype,starttime,duration,start station name,start station latitude,start station longitude
38085,Member,2016-05-02 18:10:26,34342.82,E 51 St & 1 Ave,40.754557,-73.965930
133351,Member,2016-05-05 18:17:45,24512.10,Broadway & W 29 St,40.746201,-73.988557
261833,Casual,2016-05-09 18:55:31,24216.23,Mott St & Prince St,40.723180,-73.994800
48187,Member,2016-05-02 22:11:34,23645.17,11 Ave & W 27 St,40.751396,-74.005226
488354,Casual,2016-05-14 19:20:37,22735.93,Macon St & Nostrand Ave,40.680983,-73.950048


In [87]:
narrowed_df.to_csv('narrowed_april-2016.csv', index=False)